In [2]:
import torch
from efficientnet_pytorch import EfficientNet
from PIL import Image, ImageDraw, ImageFont
import numpy as np
from pathlib import Path

import numpy as np
import cv2
import os

import time

In [3]:

#모델
MODEL_F = '/home/aa/dev_ws/vehicle-speed-estimation/trained_model3_all.pth'
# MODEL_F = '/home/aa/dev_ws/vehicle-speed-estimation/model/b0.pth'

In [4]:

#쿠다 사용 가능여부
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:

V = 0     # what version of efficientnet did you use
IN_C = 2  # number of input channels
NUM_C = 1 # number of classes to predict

In [11]:

model = EfficientNet.from_pretrained(f'efficientnet-b{V}', in_channels=IN_C, num_classes=NUM_C)
state = torch.load(MODEL_F)
model.load_state_dict(state)
model.to(device);


Loaded pretrained weights for efficientnet-b0


TypeError: Expected state_dict to be dict-like, got <class 'method'>.

In [5]:

model = torch.load(MODEL_F)

model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    2, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1))
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d((1, 1, 1, 1))
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
    

In [6]:

def draw_flow(img, flow, step=16):

    h, w = img.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1).astype(int)
    fx, fy = flow[y,x].T

    lines = np.vstack([x, y, x-fx, y-fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)

    img_bgr = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    cv2.polylines(img_bgr, lines, 0, (0, 255, 0))

    for (x1, y1), (_x2, _y2) in lines:
        cv2.circle(img_bgr, (x1, y1), 1, (0, 255, 0), -1)

    return img_bgr

In [7]:
def draw_hsv(flow):

    h, w = flow.shape[:2]
    fx, fy = flow[:, :, 0], flow[:, :, 1]

    ang = np.arctan2(fy, fx) + np.pi
    v = np.sqrt(fx*fx+fy*fy)

    hsv = np.zeros((h, w, 3), np.uint8)
    hsv[...,0] = ang*(180/np.pi/2)
    hsv[...,1] = 255
    hsv[...,2] = np.minimum(v*4, 255)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    return bgr

In [8]:
def inference(of_f):
    of = np.load(of_f)
    i = torch.from_numpy(of).to(device)
    pred = model(i)
    del i
    torch.cuda.empty_cache()
    return pred

In [9]:
OF_NPY_DIR = '/home/aa/dev_ws/opical-flow-estimation-with-RAFT/output/'

In [10]:
import sys
sys.path.append('core')

import argparse
import os
import cv2
import glob
import numpy as np
import torch
from PIL import Image
import matplotlib.pyplot as plt
from pathlib import Path

from raft import RAFT
from utils import flow_viz
from utils.utils import InputPadder

args = argparse.Namespace(images_dir="./images", output_dir="./output")
optical_model = torch.nn.DataParallel(RAFT(args))
optical_model.load_state_dict(torch.load('model/raft-things.pth'))


DEVICE = 'cuda' if torch.cuda.is_available else 'cpu'

optical_model = optical_model.module
optical_model.to(DEVICE)
optical_model.eval()


RAFT(
  (fnet): BasicEncoder(
    (norm1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (relu1): ReLU(inplace=True)
    (layer1): Sequential(
      (0): ResidualBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU(inplace=True)
        (norm1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (norm2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      )
      (1): ResidualBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU(inplace=True)
        (norm1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=Fa

In [11]:

def load_image(imfile):
    img = cv2.imread(imfile)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = np.array(img).astype(np.uint8)
    return img

def visualize_flow(flow, img_name):
    flow_image = flow_viz.flow_to_image(flow)
    cv2.imwrite(img_name, flow_image[:, :, [2, 1, 0]])


In [30]:
cap = cv2.VideoCapture('test.mp4')

with open('test.txt', mode='r', encoding='utf-8') as file:
    labels = file.read().split('\n')

label_num=0
npy_num=0
i=0


suc, prev = cap.read()
prev = cv2.resize(prev, (640,480))
prevgray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
# prev = cv2.cvtColor(prev, cv2.COLOR_BGR2RGB)

pass_even=0

while True:

    suc, next = cap.read()

    # if pass_even == 0:
    #     pass_even = 1
    # else:
    #     pass_even = 0
    #     continue

    next = cv2.resize(next, (640,480))
    gray = cv2.cvtColor(next, cv2.COLOR_BGR2GRAY)

    # next = cv2.cvtColor(next, cv2.COLOR_BGR2RGB)

    # Preprocess frames
    prev_frame_tensor = torch.from_numpy(prev).permute(2, 0, 1).unsqueeze(0).float().to(device)
    next_frame_tensor = torch.from_numpy(next).permute(2, 0, 1).unsqueeze(0).float().to(device)

    # padder = InputPadder(prev_frame_tensor.shape)
    # prev_frame_tensor, next_frame_tensor = padder.pad(prev_frame_tensor, next_frame_tensor)

    with torch.no_grad():
        flow_low, flow_up = optical_model(prev_frame_tensor, next_frame_tensor, iters=20, test_mode=True)
    

    i = flow_up
    pred = model(i)
    del i
    torch.cuda.empty_cache()

    print("raft optical : " + str(pred.item()))

    # flow_up = flow_up[0].permute(1, 2, 0).cpu().numpy()

    # flow_image = flow_viz.flow_to_image(flow_up)
    # cv2.imshow('optical', flow_image[:, :, [2, 1, 0]])

    # f = OF_NPY_DIR+str(npy_num)+'.npy'
    # npy_num += 1
    # y_hat = inference(f).item()
    # print('numpy train data'+' : '+str(y_hat))

    # end = time.time()
    # fps = 1 / (end-start)

    # print(f"{fps:.2f} FPS")

    #cv2.imshow('flow', draw_flow(gray, flow))
    #cv2.imshow('flow HSV', draw_hsv(flow))

    prev = next

    cv2.putText(next, 'estimated speed : '+str(pred.item()), org=(30,30), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255,255,255), lineType=cv2.LINE_AA)
    cv2.imshow('',next)

    # flow = cv2.calcOpticalFlowFarneback(prevgray, gray, None,
    #                                     pyr_scale=0.5, levels=3, winsize=15,
    #                                     iterations=3, poly_n=5, poly_sigma=1.2, flags=0)

    # flow = flow.transpose(2, 0, 1)
    # flow = np.expand_dims(flow, axis=0)

    # i = torch.from_numpy(flow).to(device)

    # pred = model(i)
    # del i
    # torch.cuda.empty_cache()

    # print('opencv optical : '+str(pred.item()))

    print('label : ' + str(labels[label_num]))
    label_num += 1
    

    key = cv2.waitKey(1)
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

raft optical : 22.728660583496094
label : 6.1694837
raft optical : 22.872013092041016
label : 6.1694837
raft optical : 22.52493667602539
label : 7.1728306
raft optical : 22.38260269165039
label : 7.5396805
raft optical : 22.70496368408203
label : 7.445733
raft optical : 22.267581939697266
label : 8.879872
raft optical : 22.891128540039062
label : 7.220886
raft optical : 22.491714477539062
label : 5.2527494
raft optical : 22.68191909790039
label : 4.649265
raft optical : 22.488662719726562
label : 3.6447248
raft optical : 23.019012451171875
label : 3.0718746
raft optical : 22.310630798339844
label : 3.5306754
raft optical : 22.01712417602539
label : 3.9055128
raft optical : 22.61517333984375
label : 4.4281096
raft optical : 19.38344955444336
label : 4.497137
raft optical : 22.361072540283203
label : 4.5936704
raft optical : 22.935598373413086
label : 4.613593
raft optical : 21.523563385009766
label : 5.2125244
raft optical : 17.362850189208984
label : 6.583261
raft optical : 22.98750305

In [30]:
prev_frame_tensor.shape

torch.Size([1, 3, 480, 640])

In [25]:
f

'/home/aa/dev_ws/vehicle-speed-estimation/data/train_npys/74.npy'

In [27]:
of = np.load(f)
    # i = torch.from_numpy(of).to(device)

In [29]:
of.shape

(1, 2, 480, 640)

In [33]:
flow_up.shape

torch.Size([1, 2, 480, 640])

In [45]:
next.shape

(480, 640, 3)